In [51]:
#Returns the bwt table, the bwt array and the suffix array
def BWT(seq):
    bwt_table = []
    suffix_array=[]
    bwt_array = ""
    #Create bwt table
    for index, base in enumerate(seq):
        bwt_table.append((seq[index:] + seq[:index], index))
    bwt_table.sort()
    #Create bwt array and suffix array
    for tup in bwt_table:
        bwt_array += tup[0][-1]
        suffix_array.append(tup[1])
    
    return bwt_table, bwt_array, suffix_array #Not necessary to return the bwt_table, we do not need this in the analysis

"""
Finds the Occurences dictionary
O(a,i) the number of occurrences of a in B[0,i]
Example:
BWT array = lo$oogg
Occ = {'l': [1, 1, 1, 1, 1, 1, 1], 'o': [0, 1, 1, 2, 3, 3, 3], 'g': [0, 0, 0, 0, 0, 1, 2]}
"""
def occ(array):
    occ = {}
    for base in array:
        if base == "$":
            continue
        occ[base] = []
        count = 0
        for b in array:
            if base == b:
               count += 1 
            occ[base].append(count)
    return occ

"""
Finds the C(a) dictionary
C(α) be the number of symbols in X (reference) that are 
lexicographically smaller than α
Example:
X = "googol"
α could be "l", "g" or "o"
C = {'l': 2, 'g': 0, 'o': 3}
"""
def calculateC_O(seq):
    bwt_array_table, bwt_array, S = BWT(seq)
    seq_inv = seq[::-1]
    bwt_array_table_inv, bwt_array_inv, S_inv = BWT(seq_inv)

    seq_set = set(seq) #{G,O,L,$} Find unique letter of reference / Replace with seq_set={A, T, G, C} for DNA sequences
    #subseq_set = set(subseq) #Replace with seq_set={A, T, G, C} for DNA sequences which have the 4 bases
    #bwt_array_set = set(bwt_array)
    
    #Create dictionary with the occurencies of each letter in the reference sequence
    bases_occurencies = {}
    for base in seq_set:
        if base == "$":
            continue                              #[G,O,O,G,O,L]
        bases_occurencies[base] = seq.count(base) #{G:2, O:3, L:1}

    #Create the C table
    c = {}
    for base in seq_set: #{G,O,L,$}
        count = 0
        if base == "$":
            continue
        for key, value in bases_occurencies.items():
            if base > key:
                count += value
        #C(α) be the number of symbols in X[0,n−2] that are lexicographically 
        #smaller than a
        c[base] = count


    #O(a,i) the number of occurrences of a in B[0,i]
    Occ = occ(bwt_array)
    #For the inverse of the reference
    Occ_inv = occ(bwt_array_inv)

    return c, Occ, Occ_inv, bwt_array, S



"""
Finds the D list
D(i) is the lower bound of the number of differences in W[0,i]
Example:
X = "googol"
W = "lol"
D = [0, 1, 1]
For the 0: "l" exists
For the first 1: "lo" does not exist but it could be "go" (1 error)
For the second 2: "lol" does not exist but it could be "gol" (1 error)
"""
def calculateD(subseq, c, Occ_inv, length_seq):
    k = 1
    l = length_seq
    z = 0
    D = []
    for i in range(len(subseq)):
        k = c[subseq[i]] + Occ_inv[subseq[i]][k-1] + 1
        l = c[subseq[i]] + Occ_inv[subseq[i]][l]
        if k > l:
            k = 1
            l = length_seq
            z += 1
        D.append(z)
    return D


"""
Finds the Indexes of the btw table where there is a match
"""
#inexact_recur(W, len(W)-1, 1, 0, len(X)-1, D)
def inexact_recur(subseq, i, z, k, l, D, X):
    #print(set(X[:-1]))
    
    #print("z:", z)
    #print("i:", i)

    #Update D_temp (current D value)
    if i < 0:
        D_temp = 0
    else:
        D_temp = D[i] #D=[0,1,1]
    
    #Check if the max number of differences was reached and terminate the current path
    if z < D_temp:
        #print("too many differences, terminating path\n")
        return
    #print("i:", i)

    #Check if there is a match and create a set of unique indexes
    indexes = set()
    if i < 0: #W="LOL"
        #print("{} < {}".format(z,D_temp))
        #print("Reached limit of differences. terminating path traversal, success! k=%d, l=%d\n" % (k,l))
        for m in range(k,l+1):
            indexes.add(m)
        return indexes

    final_indexes = set()
    #Check for insertions
    temp = inexact_recur(subseq, i-1, z-1, k, l, D, set_X) 
    if temp != None:
        final_indexes = final_indexes.union(temp)

    for b in set_X:
        temp_k = k
        temp_l = l
        
        #Carefull when k=0
        if k<=0:
            occur_k = 0
        else:
            occur_k = Occ[b][k-1]
        #print(l)
        #print("#char#", b)
        #print("#Occ[b][k_tmp]#", occur_k)
        #print("#Occ[b]",Occ[b])
        #print("#Occ[b][l]#",Occ[b][l])
        
        #Calculate new k
        temp_k = c[b] + occur_k + 1
        #Calculate new l
        temp_l = c[b] + Occ[b][l]
        #print("newK: {} | newL: {}".format(temp_k,temp_l))

        if temp_k <= temp_l: #If k is smaller or equal to l so the interval [k,l] is valid
            #print(b)
            #print((k,l), z)

            #Check for deletions
            temp = inexact_recur(subseq, i, z-1, temp_k, temp_l, D, set_X)
            #print("char '%s' found with k=%d, l=%d. z = %d: parent k=%d, l=%d" % (b,temp_k,temp_l,z,k,l))
            if temp != None:
                final_indexes = final_indexes.union(temp)

            #if the base is equal to the base of the subsequence, W, 
            #check if the next base in the sequence is equal to the next base in W without counting a missmatch
            if b == subseq[i]: 
                temp = inexact_recur(subseq, i-1, z, temp_k, temp_l, D, set_X)
                if temp != None:
                    final_indexes = final_indexes.union(temp)

            #if the base is NOT equal to the base of the subsequence, W, 
            #check the next base in the sequence while counting a missmatch
            else:
                temp = inexact_recur(subseq, i-1, z-1, temp_k, temp_l, D, set_X)
                if temp != None:
                    final_indexes = final_indexes.union(temp)

    return final_indexes

In [52]:
#X = input("Enter a Sequence string:")
#X = "$"
X = "googol$"
#B_table, B, S = BWT(X)

#X_inv = X[::-1]
#B_table_inv, B_inv, S_inv = BWT(X_inv)

In [53]:
#W = input("Enter a Subsequence string:")
#W = "$"
W = "lol"
c, Occ, Occ_inv, bwt_array, S = calculateC_O(X)
c

{'g': 0, 'l': 2, 'o': 3}

In [54]:
length_X = len(X)-1
D = calculateD(W, c, Occ_inv, length_X)
D

[0, 1, 1]

In [55]:
set_X = set(X[:-1])
bwt_indexes = inexact_recur(W, len(W)-1, 1, 0, len(X)-1, D, set_X)


In [56]:
X[:-1]

'googol'

In [57]:
set(X[:-1])

{'g', 'l', 'o'}

In [58]:
print("The reference sequence:", X)
print("The read:", W)
print("The suffix array:", S)
print("The BWT array:", bwt_array)
print("The BWT indexes:", bwt_indexes)

for index in bwt_indexes:
    suffix_index = S[index]
    print(X[suffix_index:suffix_index+len(W)])
#print("The allignment indexes:", )

The reference sequence: googol$
The read: lol
The suffix array: [6, 3, 0, 5, 2, 4, 1]
The BWT array: lo$oogg
The BWT indexes: {1, 5}
gol
ol$
